In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.cm as cm
import seaborn as sns
from matplotlib import gridspec
import matplotlib.animation as manimation
import matplotlib.patches as patches
import time
from collections import Counter
import sklearn.cluster as cluster
import time
import hdbscan
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
import re
import ast
#import plotly. as py
#import plotly.graph_objs as go
font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 16}

plt.rc('font', **font)
plt.rc('lines',lw=2) 
sns.set(font_scale = 2)

In [3]:
# clean up all bins, create new dataframes with contigs belonging to clusters 
# to achieve this, load all old clusters, with contig info (tSNE x,y; GC and coverage info), 
# as well as checkm info (contamination, completeness, strain heterogeneity). 

In [49]:
for expt in range(2,7):
    path = 'maindfs_withclusters/Obsidian'
    ext = '_HDB_maindf'
    method = ['_megahit','_metaSpades','_minimeta']
    for m in method:
        if expt == 2 and m == method[0]:
            maindf = pd.read_pickle(path+str(expt)+m+ext)
            maindf['BinName'] = 'Obs'+str(expt)+'_'+maindf['Genome ID'].str[0:4]+'_'+maindf['DBclusternum'].astype(str)
        else:
            loaddf = pd.read_pickle(path+str(expt)+m+ext) 
            loaddf['BinName'] = 'Obs'+str(expt)+'_'+loaddf['Genome ID'].str[0:4]+'_'+loaddf['DBclusternum'].astype(str)
            maindf = maindf.append(loaddf,ignore_index=True)
            
            
k=0
for i in maindf.columns:
    if len(i)>3 and k == 0:
        cols = [i]
        k+=1
    elif len(i)>3:
        cols = cols+ [i]

maindf_nocov = pd.DataFrame.from_records(maindf,columns=cols)

In [53]:
maindf_nocov

,BinName,CRISPR,ContigName,DBclusternum,GC Content,Gene Count,Genome,Genome ID,Lineage Class,Lineage Domain,...,Lineage Species,Read Depth,Scaffold ID,Scaffold Name,Sequence Length,k4_x,k4_y,k5_x,k5_y,well_counts
0,Obs2_mega_20,NO,SuperContig_k91_156_flag=0_multi=59.2199_len=8...,20,0.38,9,Hotspring sediment microbial communities from ...,megahit,Thermoprotei,Archaea,...,Sulfolobus tokodaii,1,3300017482 assembled Ga0186907,10001,8157,-44.886567,42.811039,26.329292,-4.279850,1
1,Obs2_mega_7,NO,SuperContig_k91_188_flag=0_multi=23.5528_len=1...,7,0.62,10,Hotspring sediment microbial communities from ...,megahit,Unassigned,Unassigned,...,Unassigned,1,3300017482 assembled Ga0186907,10002,10497,105.519840,-1.829416,8.547446,98.030581,0
2,Obs2_mega_20,NO,SuperContig_k91_204_flag=0_multi=39.5738_len=5...,20,0.36,3,Hotspring sediment microbial communities from ...,megahit,Thermoprotei,Archaea,...,Sulfolobus islandicus,1,3300017482 assembled Ga0186907,10003,5373,-37.419642,32.825094,20.230839,-9.735507,1
3,Obs2_mega_13,NO,SuperContig_k91_254_flag=0_multi=19.0169_len=6...,13,0.53,9,Hotspring sediment microbial communities from ...,megahit,Unassigned,Unassigned,...,Unassigned,1,3300017482 assembled Ga0186907,10004,6642,65.441401,-8.625271,-34.620362,73.688863,8
4,Obs2_mega_20,NO,SuperContig_k91_260_flag=0_multi=54.1035_len=5...,20,0.43,4,Hotspring sediment microbial communities from ...,megahit,Thermoprotei,Archaea,...,Unassigned,1,3300017482 assembled Ga0186907,10005,5241,-39.138862,48.874225,31.998645,-0.172044,0
5,Obs2_mega_1,NO,SuperContig_k91_263_flag=0_multi=61.1766_len=8...,1,0.44,10,Hotspring sediment microbial communities from ...,megahit,Aquificae,Bacteria,...,Hydrogenobacter thermophilus,1,3300017482 assembled Ga0186907,10006,8324,-99.865561,-30.161484,-77.588522,66.738745,0
6,Obs2_mega_9,NO,SuperContig_k91_283_flag=0_multi=56.3214_len=1...,9,0.64,15,Hotspring sediment microbial communities from ...,megahit,Thermoprotei,Archaea,...,Thermoproteus uzoniensis,1,3300017482 assembled Ga0186907,10007,12233,90.725898,-38.941702,97.700095,13.280439,0
7,Obs2_mega_12,NO,SuperContig_k91_303_flag=1_multi=14.4402_len=7...,12,0.44,9,Hotspring sediment microbial communities from ...,megahit,Unassigned,Unassigned,...,Unassigned,1,3300017482 assembled Ga0186907,10008,7633,-60.702334,75.173699,-46.458480,37.397178,0
8,Obs2_mega_7,NO,SuperContig_k91_336_flag=0_multi=36.6046_len=9...,7,0.55,10,Hotspring sediment microbial communities from ...,megahit,Unassigned,Unassigned,...,Unassigned,1,3300017482 assembled Ga0186907,10009,9619,94.858990,13.181470,6.102550,80.660300,2
9,Obs2_mega_-1,NO,SuperContig_k91_349_flag=0_multi=99.8166_len=5...,-1,0.50,4,Hotspring sediment microbial communities from ...,megahit,Unassigned,Unassigned,...,Unassigned,1,3300017482 assembled Ga0186907,10010,5635,8.779969,-56.612023,32.053010,26.510416,12


In [45]:
maindf_nocov.ContigName[29446]
maindf_nocov['Genome'][0]

'Hotspring sediment microbial communities from Obsidian Pool Yellowstone National Park Wyoming USA Obsidian 2. Combined Assembly of Gp0212715  Gp0212716 (Obsidian 2 combined assembly) (*)'

In [81]:
f = plt.figure()

gs = gridspec.GridSpec(3,2)

ax1 = f.add_subplot(gs[0,0])
sel = maindf_nocov[maindf_nocov['BinName'].str.contains('Obs2')]
cm = plt.cm.get_cmap('cool')
z = sel['GC Content']
sc = ax1.scatter(sel['k5_x'],sel['k5_y'],c=z,s=sel['Sequence Length']/5e2, alpha=0.2,cmap=cm)
plt.colorbar(sc);sc.set_clim(vmin=0.2, vmax=0.7)

ax2 = f.add_subplot(gs[0,1])
sel = maindf_nocov[maindf_nocov['BinName'].str.contains('Obs3')]
#cm = plt.cm.get_cmap('cool')
z = sel['GC Content']
sc = ax2.scatter(sel['k5_x'],sel['k5_y'],c=z,s=sel['Sequence Length']/5e2, alpha=0.2,cmap=cm)
plt.colorbar(sc)
sc.set_clim(vmin=0.2, vmax=0.7)

ax3 = f.add_subplot(gs[1,0])
sel = maindf_nocov[maindf_nocov['BinName'].str.contains('Obs4')]
#cm = plt.cm.get_cmap('cool')
z = sel['GC Content']
sc = ax3.scatter(sel['k5_x'],sel['k5_y'],c=z,s=sel['Sequence Length']/5e2, alpha=0.2,cmap=cm)
plt.colorbar(sc)
sc.set_clim(vmin=0.2, vmax=0.7)

ax4 = f.add_subplot(gs[1,1])
sel = maindf_nocov[maindf_nocov['BinName'].str.contains('Obs5')]
#cm = plt.cm.get_cmap('cool')
z = sel['GC Content']
sc = ax4.scatter(sel['k5_x'],sel['k5_y'],c=z,s=sel['Sequence Length']/5e2, alpha=0.2,cmap=cm)
plt.colorbar(sc)
sc.set_clim(vmin=0.2, vmax=0.7)

ax5 = f.add_subplot(gs[2,0])
sel = maindf_nocov[maindf_nocov['BinName'].str.contains('Obs6')]
#cm = plt.cm.get_cmap('cool')
z = sel['GC Content']
sc = ax5.scatter(sel['k5_x'],sel['k5_y'],c=z,s=sel['Sequence Length']/5e2, alpha=0.2,cmap=cm)
plt.colorbar(sc)
sc.set_clim(vmin=0.2, vmax=0.7)
f.set_figheight(25)
f.set_figwidth(20)
#plt.show()

In [86]:
maindf_nocov[maindf_nocov['BinName'].str.contains('Obs2_mini_5')]['Lineage Phylum'].value_counts()

Nanoarchaeota            19
Thermodesulfobacteria     4
Dictyoglomi               4
Unassigned                3
Euryarchaeota             3
Name: Lineage Phylum, dtype: int64

In [87]:
maindf_nocov[maindf_nocov['Lineage Phylum'].str.contains('Micrarch')]['BinName'].value_counts()

Obs6_meta_11    253
Obs6_mega_12     39
Obs6_meta_-1      8
Obs4_mega_14      5
Obs6_mega_-1      4
Obs6_meta_25      3
Obs6_mega_7       2
Obs6_meta_32      1
Obs2_mini_11      1
Obs4_mega_15      1
Obs6_meta_7       1
Obs4_meta_-1      1
Obs4_meta_13      1
Obs6_mega_31      1
Name: BinName, dtype: int64

In [88]:
maindf_nocov[maindf_nocov['BinName'].str.contains('Obs6_meta_11')]['Lineage Phylum'].value_counts()

Candidatus Micrarchaeota    253
Unassigned                    4
Name: Lineage Phylum, dtype: int64

In [187]:
def colorsel(inlist,df):
    col = sns.color_palette("deep", len(inlist)-1)
    colors = [col[x] if x >= 0 else (0.0, 0.0, 0.0) for x in df['DBclusternum']]
    
    return(colors)

def plotclusters(expt,assembly,num,sphere):
    plotdf = maindf_nocov[maindf_nocov['BinName'].str.contains(expt)]
    clustermm = plotdf[plotdf['BinName'].str.contains('mini')].copy()
    clusterms = plotdf[plotdf['BinName'].str.contains('meta')].copy()
    clustermh = plotdf[plotdf['BinName'].str.contains('mega')].copy()
    
    clistmm = np.sort(list(clustermm['DBclusternum'].unique()))
    clistms = np.sort(list(clusterms['DBclusternum'].unique()))
    clistmh = np.sort(list(clustermh['DBclusternum'].unique()))
    
    colmm = colorsel(clistmm,clustermm)
    colms = colorsel(clistms,clusterms)
    colmh = colorsel(clistmh,clustermh)
    
    f = plt.figure()
    f.set_figheight(15)
    f.set_figwidth(15)
    if assembly=='mm':
        plt.scatter(clustermm.k5_x,clustermm.k5_y,color=colmm, alpha=.45)
        plt.scatter(clusterms.k5_x,clusterms.k5_y,color='gray', alpha=.25)
        plt.scatter(clustermh.k5_x,clustermh.k5_y,color='gray', alpha=.25)
        plt.scatter(clustermm.groupby('DBclusternum').mean()[1:]['k5_x'],clustermm.groupby('DBclusternum').mean()[1:]['k5_y'],s=clustermm.groupby('DBclusternum').sum()[1:]['Sequence Length']/1e2,alpha=0)
        if sphere=="Yes":
            plt.scatter(clustermm.groupby('DBclusternum').mean()[1:]['k5_x'],clustermm.groupby('DBclusternum').mean()[1:]['k5_y'],s=clustermm.groupby('DBclusternum').sum()[1:]['Sequence Length']/1e2,alpha=0.2)
        if num=='Yes':
            al = clustermm.groupby('DBclusternum').mean()[1:]
            for clus in al.index:
                plt.text(al.loc[clus,'k5_x'],al.loc[clus,'k5_y'],clus)
        plt.xticks([]);plt.yticks([])
        plt.title(expt+' minimeta')
        
    elif assembly=='mh':
        plt.scatter(clustermm.k5_x,clustermm.k5_y,color='gray', alpha=.25)
        plt.scatter(clusterms.k5_x,clusterms.k5_y,color='gray', alpha=.25)
        plt.scatter(clustermh.k5_x,clustermh.k5_y,color=colmh, alpha=.45)
        plt.scatter(clustermh.groupby('DBclusternum').mean()[1:]['k5_x'],clustermh.groupby('DBclusternum').mean()[1:]['k5_y'],s=clustermh.groupby('DBclusternum').sum()[1:]['Sequence Length']/1e2,alpha=0,color='g')
        if sphere=="Yes":
            plt.scatter(clustermh.groupby('DBclusternum').mean()[1:]['k5_x'],clustermh.groupby('DBclusternum').mean()[1:]['k5_y'],s=clustermh.groupby('DBclusternum').sum()[1:]['Sequence Length']/1e2,alpha=0.1,color='g')
        if num=='Yes':
            al = clustermh.groupby('DBclusternum').mean()[1:]
            for clus in al.index:
                plt.text(al.loc[clus,'k5_x'],al.loc[clus,'k5_y'],clus)
        plt.xticks([]);plt.yticks([])
        plt.title(expt+' megahit')
        
    elif assembly=='ms':
        plt.scatter(clustermm.k5_x,clustermm.k5_y,color='gray', alpha=.25)
        plt.scatter(clusterms.k5_x,clusterms.k5_y,color=colms, alpha=.45)
        plt.scatter(clustermh.k5_x,clustermh.k5_y,color='gray', alpha=.25)
        plt.scatter(clusterms.groupby('DBclusternum').mean()[1:]['k5_x'],clusterms.groupby('DBclusternum').mean()[1:]['k5_y'],s=clusterms.groupby('DBclusternum').sum()[1:]['Sequence Length']/1e2,alpha=0,color='r')
        if sphere=="Yes":
            plt.scatter(clusterms.groupby('DBclusternum').mean()[1:]['k5_x'],clusterms.groupby('DBclusternum').mean()[1:]['k5_y'],s=clusterms.groupby('DBclusternum').sum()[1:]['Sequence Length']/1e2,alpha=0.1,color='r')
        if num=='Yes':
            al = clusterms.groupby('DBclusternum').mean()[1:]
            for clus in al.index:
                plt.text(al.loc[clus,'k5_x'],al.loc[clus,'k5_y'],clus)
        plt.xticks([]);plt.yticks([])
        plt.title(expt+' metaspades')
        
    elif assembly=='all':
        plt.scatter(clustermm.k5_x,clustermm.k5_y,color=colmm, alpha=.35)
        plt.scatter(clusterms.k5_x,clusterms.k5_y,color=colms, alpha=.35)
        plt.scatter(clustermh.k5_x,clustermh.k5_y,color=colmh, alpha=.35)
        plt.title(expt+' all')
        plt.xticks([]);plt.yticks([])
    #plt.show()
    if num=='Yes' and sphere=='Yes':
        f.savefig('pyplots/clusternumbers/contigplots/'+expt+'_'+assembly+'_withnum_sphere')
    elif sphere=='Yes':
        f.savefig('pyplots/clusternumbers/contigplots/'+expt+'_'+assembly+'_withsphere')
    elif num=='Yes':
        f.savefig('pyplots/clusternumbers/contigplots/'+expt+'_'+assembly+'_withnum')
    else:
        f.savefig('pyplots/clusternumbers/contigplots/'+expt+'_'+assembly)
    #return(colors)

In [188]:
# plot and save all
for expt in range(2,7):
    expt = 'Obs'+str(expt)
    for typ in ['mm','mh','ms']:
        plotclusters(expt,typ,'no','no')
        plotclusters(expt,typ,'no','Yes')
        plotclusters(expt,typ,'Yes','no')
        plotclusters(expt,typ,'Yes','Yes')
    plotclusters(expt,'all','no','no')


/usr/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [244]:
for j in range(2,7):
    CR_Obs = pd.DataFrame.from_csv('20170815_cluster_red_Obs'+str(j)+'.csv',index_col=None)
    CR_Obs['sample'] = j
    if j==2:
        mergeClust = CR_Obs.copy()
    else:
        mergeClust = mergeClust.append(CR_Obs,ignore_index=True)
    
mergeClust = mergeClust.fillna(-1).astype(int)
mergeClust['sample'] = ['Obs'+str(s) for s in mergeClust['sample']]
mergeClust.index = ['M0'+str(m) if m<10 else 'M'+str(m) for m in mergeClust.index]
mergeClust[mergeClust['sample']=='Obs3']

checkmmeta = pd.read_table('Checkm/obsidian_all_checkm/'+'outputfile.txt')
checkmmeta = checkmmeta.set_index('Bin Id')
mli = list(mergeClust[mergeClust['sample']=='Obs3'].index)
checkmmetaobs3 = checkmmeta[checkmmeta.index.str.contains('Obs3')].append(checkmmeta[checkmmeta.index.isin(mli)])

In [259]:
def plot_cluster_QC(expt):
    mli = list(mergeClust[mergeClust['sample']==expt].index)
    checkmmetaobs = checkmmeta[checkmmeta.index.str.contains(expt)].append(checkmmeta[checkmmeta.index.isin(mli)])


    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(25)
    gs = gridspec.GridSpec(2,8)

    ax1 = f.add_subplot(gs[0:,0])
    checkmmetaobs[checkmmetaobs.index.str.contains('mini')].plot.barh(y=['Completeness','Contamination','Strain heterogeneity'],ax=ax1)
    plt.ylabel('');ax1.legend_.remove();plt.xlim(0,100)
    #plt.legend(bbox_to_anchor=(1.05, 1),loc=2)
    ax2 = f.add_subplot(gs[0:,2])
    checkmmetaobs[checkmmetaobs.index.str.contains('meta')].plot.barh(y=['Completeness','Contamination','Strain heterogeneity'],ax=ax2)
    plt.ylabel('');ax2.legend_.remove();plt.xlim(0,100)
    ax3 = f.add_subplot(gs[0:,4])
    checkmmetaobs[checkmmetaobs.index.str.contains('mega')].plot.barh(y=['Completeness','Contamination','Strain heterogeneity'],ax=ax3)
    plt.ylabel('');ax3.legend_.remove();plt.xlim(0,100)

    ax4 = f.add_subplot(gs[0:,6])
    checkmmetaobs[checkmmetaobs.index.str.contains('M')].plot.barh(y=['Completeness','Contamination','Strain heterogeneity'],ax=ax4)
    plt.ylabel('');plt.xlim(0,100)
    plt.legend(bbox_to_anchor=(1.05, 1),loc=2)

    f.savefig('pyplots/clusternumbers/contigplots/checkmQC_2_'+expt)
    
for ex in range(2,7):
    li = 'Obs'+str(ex)
    plot_cluster_QC(li)

In [242]:
checkmmeta[checkmmeta.index.isin(mli)]

,Marker lineage,# genomes,# markers,# marker sets,0,1,2,3,4,5+,Completeness,Contamination,Strain heterogeneity
Bin Id,,,,,,,,,,,,,


In [243]:
mli

['mini', 'meta', 'mega', 'sample']

In [257]:
mergeClust[mergeClust['sample']=='Obs6']

,mini,meta,mega,sample
M65,-1,0,0,Obs6
M66,-1,1,1,Obs6
M67,-1,2,2,Obs6
M68,-1,13,3,Obs6
M69,-1,4,4,Obs6
M70,-1,17,5,Obs6
M71,-1,7,7,Obs6
M72,-1,11,12,Obs6
M73,-1,9,14,Obs6
M74,-1,19,19,Obs6


In [251]:
checkmmeta

,Marker lineage,# genomes,# markers,# marker sets,0,1,2,3,4,5+,Completeness,Contamination,Strain heterogeneity
Bin Id,,,,,,,,,,,,,
M00,k__Bacteria (UID203),5449,102,56,3,8,91,0,0,0,96.27,88.23,100.00
M01,k__Archaea (UID2),207,147,106,5,45,96,1,0,0,95.75,67.96,98.99
M02,k__Archaea (UID2),207,148,106,17,22,73,36,0,0,85.78,93.46,98.90
M03,k__Bacteria (UID203),5449,104,58,77,25,2,0,0,0,30.96,2.59,100.00
M04,k__Archaea (UID146),59,174,136,84,75,13,2,0,0,46.94,10.66,36.84
M05,root (UID1),5656,56,24,0,0,56,0,0,0,100.00,100.00,100.00
M06,c__Thermoprotei (UID148),41,245,158,136,76,33,0,0,0,43.28,13.38,100.00
M07,c__Thermoprotei (UID148),41,245,158,19,166,59,1,0,0,89.24,31.25,95.16
M08,k__Archaea (UID2),207,149,107,5,8,73,61,2,0,95.79,127.85,97.01


In [258]:
alt5

NameError: name 'alt5' is not defined